# Taking Geospatial Data Analytics to the Next Level with Panel, DuckDB and MapLibre

### Introduction
In this blog post, we will explore how to prepare and query data using DuckDB. Next, we will create a custom component from scratch with Python Panel and integrate it with MapLibreGL JS. Finally, we will demonstrate how to dynamically update map data based on the zoom level, step by step.

### Data Analysis
We will use DuckDB to explore the POI (Point of Interest) data shared by Foursquare as open data. We will query and retrieve data for Turkey and generate H3 indices for this data using the DuckDB H3 extension.

### Project Setup

We will create the project structure using Python Uv. We will also install all the required packages to set up the working environment. We install the Uv package manager using the following code:

curl -LsSf https://astral.sh/uv/install.sh | sh

We create the project using the following code:

uv init open-poi
cd open-poi

To set up the virtual environment, we use the following command:

uv venv --python 3.11

We install the required packages for our project using the following commands:

uv add "duckdb>=1.1.3" "panel>=1.5.5"

To include Jupyter Notebook in our development environment, we install it using the following command:

uv add jupyterlab --dev

We create a notebook directory in our project folder and add a notebook file named “poi-data.ipynb” inside it. To install the spatial and H3 extensions for DuckDB:




In [1]:
import duckdb
import os


db_dir = '/Users/ulrike_imac_air/projects/maps/GeospatialDataAnalysisUsingDuckDB/open-poi/data'
db_path = os.path.join(db_dir, 'poi.duckdb')
db = duckdb.connect(db_path, read_only=False)
db.sql("""
INSTALL spatial;
INSTALL h3 FROM community;
LOAD h3;
LOAD spatial;
""")

To load the data shared by Foursquare into DuckDB, use the following SQL queries. These create two separate tables: categories for all categories and places for data filtered to only include locations in Turkey (country = ‘TR’):

In [2]:
db.sql("""
DROP TABLE IF EXISTS categories;
""")

db.sql("""
CREATE TABLE categories AS  
  SELECT * 
  FROM read_parquet('s3://fsq-os-places-us-east-1/release/dt=2025-01-10/categories/parquet/*.zstd.parquet');
""")

In [3]:
db.sql("""
DROP TABLE IF EXISTS places;
""")

db.sql("""
CREATE TABLE places AS  
  SELECT * 
  FROM read_parquet('s3://fsq-os-places-us-east-1/release/dt=2025-01-10/places/parquet/*.zstd.parquet')
  WHERE country = 'LU';
""")

To convert the geom column to a geometry type, you can use the following SQL query. This uses the ST_GeomFromWKB function in DuckDB to transform the data in the geom column from WKB (Well-Known Binary) format to geometry type:

In [4]:
db.sql("""
ALTER TABLE places 
ALTER COLUMN geom 
SET DATA TYPE GEOMETRY USING ST_GeomFromWKB(geom);
""")

BinderException: Binder Error: No function matches the given name and argument types 'ST_GeomFromWKB(GEOMETRY)'. You might need to add explicit type casts.
	Candidate functions:
	ST_GeomFromWKB(WKB_BLOB) -> GEOMETRY
	ST_GeomFromWKB(BLOB) -> GEOMETRY

LINE 4: SET DATA TYPE GEOMETRY USING ST_GeomFromWKB(geom);
                                     ^

To generate different H3 resolutions (6, 7, 8, 9, 10, 11, 12) based on the zoom level, use the following code. This code adds new H3 columns to the places table for each resolution and populates them with the appropriate H3 indices:

In [5]:
for i in range(6, 13):
    db.sql(f"""
    ALTER TABLE places 
    ADD COLUMN h{i} BIGINT;

    UPDATE places 
    SET h{i} = h3_latlng_to_cell(latitude, longitude, {i});
    """)

To view the first 10 rows of the places table, use the following query:

In [6]:
db.sql("""
select * from places limit 10;
""")

┌──────────────────────────┬──────────────────────────────────────────────────────────────┬────────────────────┬───────────────────┬──────────────────────┬──────────────────┬────────────────────────┬──────────┬──────────────┬───────────┬─────────┬─────────┬──────────────┬────────────────┬─────────────┬─────────────┬──────────────────────────────────────────────────┬───────────────┬─────────────────┬───────────┬─────────────────┬────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────┬────────

Creating a Custom Component
We will build a custom component from scratch using Panel. This component will enable interactive map rendering and dynamic data updates with MapLibreGL JS.

Create a file named app.py in your project directory and add the following starter code. This code imports the essential packages needed for the application.


In [ ]:
# app.py

import json
import panel as pn
import duckdb
from panel.custom import JSComponent
import param
from pathlib import Path

To connect to the DuckDB database and load the necessary extensions:

In [ ]:
db = duckdb.connect(database='data/poi.duckdb', read_only=True)
db.sql("""
    INSTALL spatial;
    INSTALL h3 FROM community;
    LOAD h3;
    LOAD spatial;
""")

Below is the function that retrieves data from the database based on the zoom level (z) and bounding box (bounds) and converts it into GeoJSON format:

In [ ]:
def generate_geojson(z: int, bounds: dict = None):
    """
    Generates a GeoJSON FeatureCollection from database query results.
    Args:
        z (int): The zoom level for H3 hexagons.
        bounds (dict, optional): A dictionary containing the bounding box coordinates with keys 'minx', 'maxx', 'miny', and 'maxy'.
    Returns:
        dict: A GeoJSON FeatureCollection containing the H3 hexagons and their associated properties.
    """

    q = f"""
    SELECT h3_h3_to_string(h{z}) as h3, 
        COUNT(*) as count, 
        NTILE(10) OVER (ORDER BY count) as q10,
        ST_AsGeoJSON(ST_GeomFromText(h3_cell_to_boundary_wkt(h{z}))) as geojson
    FROM places
        WHERE bbox.xmin >= {bounds['minx']} AND bbox.xmax <= {bounds['maxx']}
            AND bbox.ymin >= {bounds['miny']} AND bbox.ymax <= {bounds['maxy']} 
            and h{z} is not null
    GROUP BY h{z}
    """

    result = db.sql(q).df().to_dict(orient='records')
    # Convert the list of JSON objects to a GeoJSON FeatureCollection
    return {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": json.loads(row["geojson"]),
                "properties": {
                    "h3": row["h3"],
                    "count": row["count"],
                    "q10": row["q10"]
                }
            }
            for row in result
        ]
    }

    The Map class is designed to create an interactive map using MapLibre GL. This class retrieves GeoJSON data from the database based on the zoom level and bounding box, dynamically updating the map.

Class Attributes:

• data_geo: A param.Dict property that holds GeoJSON data. This data is dynamically updated to reflect what is displayed on the map.

• _esm: Path to the JavaScript file (src/map.js) that powers the MapLibre GL map component.

• __css__: Loads the required CSS file for the map’s styling. This file provides the visual styles for the MapLibre GL map.

_handle_msg: A method that processes messages from the map component. The method passes this information to the generate_geojson function and updates the data_geo property with the returned GeoJSON data.